In [63]:
import os
import sys

import numpy as np
import scipy.io

import sys
sys.path.append("../../dpc/")
from util.point_cloud import point_cloud_distance
from util.simple_dataset import Dataset3D
from util.app_config import config as app_config
from util.tools import partition_range, to_np_object
from util.quaternion import quaternion_rotate
from util.euler import ypr_from_campos

import torch
from torch.utils.tensorboard import SummaryWriter
from models import model_pc_to as model_pc
from util.system import setup_environment
from run.ShapeRecords import ShapeRecords

import pickle
import pdb

In [64]:
def compute_distance(cfg, source_np, target_np):
    """
    compute projection from source to target
    """
    num_parts = cfg.pc_eval_chamfer_num_parts
    partition = partition_range(source_np.shape[0], num_parts)
    min_dist_np = np.zeros((0,))
    idx_np = np.zeros((0,))
    source_pc = torch.from_numpy(source_np).cuda()
    target_pc = torch.from_numpy(target_np).cuda()
    for k in range(num_parts):
        r = partition[k, :]
        src = source_pc[r[0]:r[1]]
        _, min_dist, min_idx = point_cloud_distance(src, target_pc)
        min_dist_0_np = min_dist.cpu().numpy()
        idx_0_np = min_idx.cpu().numpy()
        min_dist_np = np.concatenate((min_dist_np, min_dist_0_np), axis=0)
        idx_np = np.concatenate((idx_np, idx_0_np), axis=0)

    return min_dist_np, idx_np

In [65]:

def get_group(pos):

    divs = 2
    scale = divs/2
    yaw, pitch, roll = ypr_from_campos(pos[0], pos[1], pos[2])
    yaw = yaw + np.pi

    # get everything from 0 to 2*pi
    yaw = yaw%(2*np.pi)+0.00000001
    pitch = pitch%(2*np.pi)+0.00000001
    roll = roll%(2*np.pi) + 0.00000001

    q1 = np.ceil(scale*yaw/np.pi)-1
    q2 = np.ceil(scale*pitch/np.pi)-1
    q3 = np.ceil(scale*roll/np.pi)-1

    return q1*np.square(divs)+q2*divs+q3

In [66]:
def run_eval():
    divs = 2
    cfg = app_config
    exp_dir = cfg.checkpoint_dir
    num_views = cfg.num_views
    eval_unsup = cfg.eval_unsupervised_shape
    dataset_folder = cfg.inp_dir

    gt_dir = os.path.join(cfg.gt_pc_dir, cfg.synth_set)

    save_pred_name = "{}_{}".format(cfg.save_predictions_dir, cfg.eval_split)
    save_dir = os.path.join(exp_dir, cfg.save_predictions_dir)

    if eval_unsup:
        reference_rotation = scipy.io.loadmat("{}/final_reference_rotation.mat".format(exp_dir))["rotation"]

    dataset = ShapeRecords(dataset_folder, cfg, 'test')

    if cfg.models_list:
        model_names = parse_lines(cfg.models_list)
    else:
        model_names = dataset.file_names
    num_models = len(model_names)

    # making groups for samples and views according to 8 groups of yaw, pitch, roll
    chamfer_dict = {}
    images_dict = {}
    all_images = {}
    for j in range(np.power(divs, 3)):
        chamfer_dict[j] = np.zeros((0,2))
        images_dict[j] = np.zeros((0,3,128,128))
        all_images[j] = np.zeros((0,3,128,128))

    for k in range(num_models):
        sample = dataset.__getitem__(k)

        print("{}/{}".format(k, num_models))
        print(model_names[k])

        gt_filename = "{}/{}.mat".format(gt_dir, model_names[k]).replace('_features.p', '')
        mat_filename = "{}/{}_pc.pkl".format(save_dir, model_names[k])

        if not os.path.isfile(gt_filename) or not os.path.isfile(mat_filename):
            continue

        with open(mat_filename, 'rb') as handle:
            data = pickle.load(handle)
        all_pcs = np.squeeze(data["points"])
        if "num_points" in data:
            all_pcs_nums = np.squeeze(data["num_points"])
            has_number = True
        else:
            has_number = False
        obj = scipy.io.loadmat(gt_filename)
        Vgt = obj["points"]

        for i in range(num_views):
            chamfer_dists_current = np.zeros((2), dtype=np.float64)

            pred = all_pcs[i, :, :]
            if has_number:
                pred = pred[0:all_pcs_nums[i], :]

            if eval_unsup:
                pred = np.expand_dims(pred, 0)
                pred = quaternion_rotate(torch.from_numpy(pred).cuda(),
                                         torch.from_numpy(reference_rotation).cuda()).cpu().numpy()
                pred = np.squeeze(pred)

            pred_to_gt, idx_np = compute_distance(cfg, pred, Vgt)
            gt_to_pred, _ = compute_distance(cfg, Vgt, pred)
            chamfer_dists_current[0] = np.mean(pred_to_gt)
            chamfer_dists_current[1] = np.mean(gt_to_pred)

            is_nan = np.isnan(pred_to_gt)
            assert (not np.any(is_nan))

            campos = sample['cam_pos'][i]
            images = sample['image'][i]
            g = get_group(campos)

            chamfer_dict[g] = np.concatenate((chamfer_dict[g], np.expand_dims(chamfer_dists_current, 0)))
            images_dict[g] = np.concatenate((images_dict[g], np.expand_dims(images,0)))
            all_images[i] = np.concatenate((all_images[i],np.expand_dims(images,0)))
            print(i, ":", chamfer_dists_current)

        # current_mean = np.mean(chamfer_dists_current, 0)
        # print("total:", current_mean)

    from sklearn.decomposition import PCA
    for k in range(len(all_images)):
        images_val = all_images[k].reshape(all_images[k].shape[0],-1)
        pca = PCA()
        pca_image = pca.fit_transform(images_val)
        pca_image = pca_image.reshape((-1,32,32))
        all_images[k] = pca_image

    pdb.set_trace()
        #from sklearn.cluster.KMeans(n_clusters=6,init='k-means++',random_state=0).fit(X)





    for key in chamfer_dict:
        print(key, np.mean(chamfer_dict[key],0)*100)

    from sklearn.decomposition import PCA
    pca_dict = {}
    for key in images_dict:
        images_val = images_dict[key].reshape(images_dict[key].shape[0],-1)
        pca = PCA()
        pca_image = pca.fit_transform(images_val)
        pca_image.reshape((-1,32,32,1))
        pca_dict[key] = pca_image


    # scipy.io.savemat(os.path.join(exp_dir, "chamfer_{}.mat".format(save_pred_name)),
    #                  {"chamfer": chamfer_dists,
    #                   "model_names": to_np_object(model_names)})
    #
    # file = open(os.path.join(exp_dir, "chamfer_{}.txt".format(save_pred_name)), "w")
    # file.write("{} {}\n".format(final[0], final[1]))
    # file.close()

        

In [67]:
    cfg = app_config

    setup_environment(cfg)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    train_dir = cfg.checkpoint_dir

    split_name = "eval"
    dataset_folder = cfg.inp_dir
    
    dataset = ShapeRecords(dataset_folder, cfg, 'test')
    dataset_loader = torch.utils.data.DataLoader(dataset,
                                                 batch_size=cfg.batch_size, shuffle=cfg.shuffle_dataset,
                                                 num_workers=4, drop_last=True)
    
    

Initialized dataset test with size 1355


In [ ]:
    divs = 2
    cfg = app_config
    exp_dir = cfg.checkpoint_dir
    num_views = cfg.num_views
    eval_unsup = cfg.eval_unsupervised_shape
    dataset_folder = cfg.inp_dir

    gt_dir = os.path.join(cfg.gt_pc_dir, cfg.synth_set)

    save_pred_name = "{}_{}".format(cfg.save_predictions_dir, cfg.eval_split)
    save_dir = os.path.join(exp_dir, cfg.save_predictions_dir)

    if eval_unsup:
        reference_rotation = scipy.io.loadmat("{}/final_reference_rotation.mat".format(exp_dir))["rotation"]

    dataset = ShapeRecords(dataset_folder, cfg, 'test')

    if cfg.models_list:
        model_names = parse_lines(cfg.models_list)
    else:
        model_names = dataset.file_names
    num_models = len(model_names)

    # making groups for samples and views according to 8 groups of yaw, pitch, roll
    chamfer_dict = {}
    images_dict = {}
    all_images = {}
    for j in range(np.power(divs, 3)):
        chamfer_dict[j] = np.zeros((0,2))
        images_dict[j] = np.zeros((0,3,128,128))
        all_images[j] = np.zeros((0,3,128,128))

    for k in range(64):
        sample = dataset.__getitem__(k)

        print("{}/{}".format(k, num_models))
        print(model_names[k])

        gt_filename = "{}/{}.mat".format(gt_dir, model_names[k]).replace('_features.p', '')
        mat_filename = "{}/{}_pc.pkl".format(save_dir, model_names[k])

        if not os.path.isfile(gt_filename) or not os.path.isfile(mat_filename):
            continue

        with open(mat_filename, 'rb') as handle:
            data = pickle.load(handle)
        all_pcs = np.squeeze(data["points"])
        if "num_points" in data:
            all_pcs_nums = np.squeeze(data["num_points"])
            has_number = True
        else:
            has_number = False
        obj = scipy.io.loadmat(gt_filename)
        Vgt = obj["points"]

        for i in range(num_views):
            chamfer_dists_current = np.zeros((2), dtype=np.float64)

            pred = all_pcs[i, :, :]
            if has_number:
                pred = pred[0:all_pcs_nums[i], :]

            if eval_unsup:
                pred = np.expand_dims(pred, 0)
                pred = quaternion_rotate(torch.from_numpy(pred).cuda(),
                                         torch.from_numpy(reference_rotation).cuda()).cpu().numpy()
                pred = np.squeeze(pred)

            pred_to_gt, idx_np = compute_distance(cfg, pred, Vgt)
            gt_to_pred, _ = compute_distance(cfg, Vgt, pred)
            chamfer_dists_current[0] = np.mean(pred_to_gt)
            chamfer_dists_current[1] = np.mean(gt_to_pred)

            is_nan = np.isnan(pred_to_gt)
            assert (not np.any(is_nan))

            campos = sample['cam_pos'][i]
            images = sample['image'][i]
            g = get_group(campos)

            chamfer_dict[g] = np.concatenate((chamfer_dict[g], np.expand_dims(chamfer_dists_current, 0)))
            images_dict[g] = np.concatenate((images_dict[g], np.expand_dims(images,0)))
            all_images[(k*num_views) +i] = np.concatenate((all_images[i],np.expand_dims(images,0)))
            #print(i, ":", chamfer_dists_current)

        # current_mean = np.mean(chamfer_dists_current, 0)
        # print("total:", current_mean)        

Initialized dataset test with size 1355
0/1355
cca975f4a6a4d9e9614871b18a2b1957_features.p
1/1355
47eff1e5c09cec7435836c728d324152_features.p
2/1355
b2d91caf3615a9a7f51f77a6d7299806_features.p
3/1355
720bc3f0677a75fb2f134420f7b0d7e6_features.p
4/1355
f7b52f9c95ebaf3ff91663a74ccd2338_features.p
5/1355
ac041c720dc66c06d1c83bc8b134e4d8_features.p
6/1355
d350f40e5f04360565ba78ad9601cf1b_features.p
7/1355
5283a98b5c693e64ebefe6b1d594ad2e_features.p
8/1355
f595abef9bc7320944b2fa2cac0778f5_features.p
9/1355
1c2b230840baac57df3c82bbe2e3ca02_features.p
10/1355
9a711bb7070ae88de948e3d64826c640_features.p
11/1355
3dac0fd995c747ab336e8bf2357c029a_features.p
12/1355
bd0fab2e72b445bd1e722bceee6e83aa_features.p
13/1355
681f4302a8bd9698f4b6538438a0b930_features.p
14/1355
a33e91aea2b278e0c5ee4aa82bd336ce_features.p
15/1355
5ab67b30e17a2ed1c862eec8232fff1e_features.p
16/1355
8f1a661dc1048e36297936c81e7f6629_features.p
17/1355
270b86f1c341d7fc98d5fc0473d00a1c_features.p
18/1355
67d3a3eceaada710e400ee8c5c

157/1355
4c1777173111f2e380a88936375f2ef4_features.p
158/1355
6dddf2b95ca09bf5febad4f49b26ec52_features.p
159/1355
da264a41dd8fdd655357c338ec9641_features.p
160/1355
751342c7107c86f0fa9e9ca2723d9727_features.p
161/1355
736630a749935489f78c99506bdaf97d_features.p
162/1355
9dc1371b6989479c9c0ca344f487323e_features.p
163/1355
32408a694ac56775ce62834dbceac22f_features.p
164/1355
d417795442cb0f75593ebeeedbff73b_features.p
165/1355
450cd2f81b38b6fc7ec728abdd798547_features.p
166/1355
893e3f15f023e9abdba08b82bf5c2f3_features.p
167/1355
4610136087e41261f1d6726679b21945_features.p
168/1355
6932261b5c17f0de4733824eae5cd9ae_features.p
169/1355
69aeeebc9a686862cbacd8aaf9fd8072_features.p
170/1355
c8860d4563be0f34f2b7580a739cd4d5_features.p
171/1355
7a338e9b13ae5da3245e874042d8b8db_features.p
172/1355
63f6bb7487f81a03bdfa5e0753fa3240_features.p
173/1355
65da0b239837b0f3f3ee5e8ea028d6_features.p
174/1355
b2d6c064af0c735f1783a44a88d6274_features.p
175/1355
ed40add41826f68ab76e29c9c43bc7aa_features.p


312/1355
68747a56aba6ee318677364bd64735c4_features.p
313/1355
c55077d0b8fd893fdc4d371eb87a65ca_features.p
314/1355
ed751e0c20f48b3226fc87e2982c8a2b_features.p
315/1355
a1adf14b24469f8835acfdef2ece21c0_features.p
316/1355
6be3e544deece8441191025061735ea3_features.p
317/1355
8f2cc8ff68f3208ec935dd3bb5739fc_features.p
318/1355
972d675d3ce3555d492d9da2668ec34c_features.p
319/1355
468b7e16a29ce746bd08524bcb66a009_features.p
320/1355
4447ee47cd2333f6349bb1cbbf9a4206_features.p
321/1355
4a24652fbf2bed7e93583c67df8faf1_features.p
322/1355
5f562c9c8f0df90fd2f33aaf794b5932_features.p
323/1355
878eee28ccc28b2e4c7384dbb75cab0d_features.p
324/1355
48f2c6ee2c8ea7538cb22acd3160c793_features.p
325/1355
b4827a7b42d68aadbeab82a04eef83b6_features.p
326/1355
c73d63f9080e84a91b17743c18fb63dc_features.p
327/1355
b3ef15af53eff822423da7f4f4231bf7_features.p
328/1355
80bad2242183a77df69c1bc654d8fbbd_features.p
329/1355
30488c6d05b4848388df69d6c56c6047_features.p
330/1355
4623efb456cc6070820018801b237b3d_featur

467/1355
3fc1e991fee82e42713decb1a0563b12_features.p
468/1355
4304100faeef130337f67f4322e8d3d0_features.p
469/1355
629117f474a4bd482d7c2f5a9f32d6bd_features.p
470/1355
5c427d9fe3c76b356c07fcbf2661d108_features.p
471/1355
d0c6af0d44cc068523c1b1df1d480bd3_features.p
472/1355
1d0abb4d48b46e3f492d9da2668ec34c_features.p
473/1355
d9aa42439dc6f1b1c63608961819b578_features.p
474/1355
103b75dfd146976563ed57e35c972b4b_features.p
475/1355
1b7bef12c554c1244c686b8271245d1b_features.p
476/1355
cc5dea482768651ed0e93cdb469eb63_features.p
477/1355
8bd8c34158a3aa04f91663a74ccd2338_features.p
478/1355
3928ff39e32dbddbf91663a74ccd2338_features.p
479/1355
44689408a7d879a664d56d7ed347e929_features.p
480/1355
aa93247a7fa69991e074517a246f1e65_features.p
481/1355
5e44499940fc3dbe57a847db7547c1f3_features.p
482/1355
33aaad494817a6f4ab705559ec99536f_features.p
483/1355
bdaaebf065b112da492d9da2668ec34c_features.p
484/1355
b1908805f05c6f47f51f77a6d7299806_features.p
485/1355
491eb8f7028d06152966622c3d8ee695_featu

622/1355
c13d5f3639f427587fa29a3ea424473_features.p
623/1355
8b552c23c064b96179368d1198f406e7_features.p
624/1355
1bb81d54471d7c1df51f77a6d7299806_features.p
625/1355
a93853bdb9b378dd50c2ec40da48fd9d_features.p
626/1355
60790035c8126a677645b4fdaedbc34_features.p
627/1355
a273d2fa5244c74f654ea6737b0d3597_features.p
628/1355
79317a7ee8c26288ce265ed390193062_features.p
629/1355
c86cfe147872280463626070a93463cf_features.p
630/1355
b11c616a19d702cdd7f7a4c4609b0913_features.p
631/1355
6d78523e939aedd482e23b5f9fd46e1e_features.p
632/1355
2f2da13322d30ccaf4b6538438a0b930_features.p
633/1355
5e598a82d584211d2681d406cc3868fb_features.p
634/1355
436be9df788b706fb40f0ac0fb9a650d_features.p
635/1355
5a30a8edad307d8b04cb542e2c50eb4_features.p
636/1355
331bfb4cc56f09e201cbed672acdcde_features.p
637/1355
7fc00af759f4382c48d90eef8384210_features.p
638/1355
f1a5a9aec0b63b91f487e71bd57b6e0c_features.p
639/1355
4ed0eb4f4e7ea0952efff55e0bb2e42d_features.p
640/1355
afaa508214b7515a161d68600952d375_features.

777/1355
5a95f4fbfbfc55bd93768e7b9b1eabf_features.p
778/1355
4f7e9b96404fe19af37f22e93a631b3a_features.p
779/1355
ce387113cca9c0754d33a7fd9c37a7d2_features.p
780/1355
2f4b0d6e2faf22d68c18ac3fe1584a6_features.p
781/1355
75f32479a35f39e9f43d965e3ff9c5b6_features.p
782/1355
4ae919ca418d3625d30ba6a673002f3e_features.p
783/1355
c5be8052dddf9e92283b00891f680579_features.p
784/1355
b88d8b5e5fbee4fa8336a02debb9923b_features.p
785/1355
3eb9c5fb2a7ab54d490ad276cd2af3a4_features.p
786/1355
fb056f0260d4855d36c8b9bf2576aca2_features.p
787/1355
465c2cc6917e02fec681404257d94ad9_features.p
788/1355
6eebd118abb1b4146d60aef5fe7e2185_features.p
789/1355
8e4269ac16e13433d701c68e1045b606_features.p
790/1355
d374912c3a9fca96c141a04b2a487fd9_features.p
791/1355
bb90094030f369e4305a3b2fd9173d6f_features.p
792/1355
fcd8163b26250f499fe49e672436bc45_features.p
793/1355
d9dc32f36e03d9f5ea65a95e759e03d9_features.p
794/1355
1ef99f4e735ceabad97996b11dc03f35_features.p
795/1355
7309610420ae20c930ddb6ead95f49cc_featur

### Obtain means of every cluster while grouping by Cam Pose

In [50]:
for key in chamfer_dict:
        print(key, np.mean(chamfer_dict[key],0)*100)

0 [3.18985801 2.85600455]
1 [nan nan]
2 [3.72138359 3.1946595 ]
3 [nan nan]
4 [3.523689   3.11041036]
5 [nan nan]
6 [3.99512329 3.48821917]
7 [nan nan]


/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


### PCA of all images

In [51]:
from sklearn.decomposition import PCA

In [52]:
all_images[0].shape

(1, 3, 128, 128)

In [53]:
out_all_images= np.zeros((1,128,32))
for k in range(len(all_images)):
        print("Index:",k)
        if not all_images[k].any():
            print("No objects grouped under group",k)
            del all_images[k]
        else:
            images_val = all_images[k].reshape(all_images[k].shape[-1],-1)
            pca = PCA(32)
            pca_image = pca.fit_transform(images_val)
            print(pca_image)
            pca_image = pca_image.reshape((-1,32))
            all_images[k] = pca_image
            print(np.expand_dims(all_images[k],0).shape)
            print(out_all_images.shape)
            out_all_images=np.concatenate((out_all_images,np.expand_dims(all_images[k],0)),axis=0)

Index: 0
[[-3.60665865 -0.91348007 -0.2612595  ...  0.00487705  0.00936874
   0.00644949]
 [-3.60665865 -0.91348007 -0.2612595  ...  0.00487705  0.00936874
   0.00644949]
 [-3.60665865 -0.91348007 -0.2612595  ...  0.00487705  0.00936874
   0.00644949]
 ...
 [-3.60665865 -0.91348007 -0.2612595  ...  0.00487705  0.00936874
   0.00644949]
 [-3.60665865 -0.91348007 -0.2612595  ...  0.00487705  0.00936874
   0.00644949]
 [-3.60665865 -0.91348007 -0.2612595  ...  0.00487705  0.00936874
   0.00644949]]
(1, 128, 32)
(1, 128, 32)
Index: 1
[[-3.90773767e+00 -5.30247950e-01  9.83402504e-01 ... -1.57633792e-03
   1.56232929e-05  5.88374664e-03]
 [-3.90773767e+00 -5.30247950e-01  9.83402504e-01 ... -1.57633792e-03
   1.56232929e-05  5.88374664e-03]
 [-3.78077615e+00 -4.47335732e-01  8.11363401e-01 ... -9.19163689e-02
   3.91344253e-02 -2.80285798e-02]
 ...
 [-3.90773767e+00 -5.30247950e-01  9.83402504e-01 ... -1.57633792e-03
   1.56232929e-05  5.88374664e-03]
 [-3.90773767e+00 -5.30247950e-01  9.83

[[-4.21725556e+00 -3.90573987e-01 -9.30372758e-01 ... -4.73303672e-02
  -2.69699806e-02 -6.68773299e-03]
 [-2.76763798e-01 -3.43627784e-01 -2.13552390e+00 ... -2.50999246e-01
   1.93797354e-01  3.54951196e-01]
 [ 8.66507467e+00 -3.12067327e+00 -8.22534963e-01 ... -1.94521128e-01
  -3.94340957e-01 -2.01852576e-01]
 ...
 [-4.21725556e+00 -3.90573987e-01 -9.30372758e-01 ... -4.73303672e-02
  -2.69699806e-02 -6.68773299e-03]
 [-4.21725556e+00 -3.90573987e-01 -9.30372758e-01 ... -4.73303672e-02
  -2.69699806e-02 -6.68773299e-03]
 [-4.21725556e+00 -3.90573987e-01 -9.30372758e-01 ... -4.73303672e-02
  -2.69699806e-02 -6.68773299e-03]]
(1, 128, 32)
(15, 128, 32)
Index: 15
[[-4.8482711   0.10107895 -1.12880599 ... -0.03815857 -0.04307037
  -0.02108878]
 [-4.8482711   0.10107895 -1.12880599 ... -0.03815857 -0.04307037
  -0.02108878]
 [-4.8482711   0.10107895 -1.12880599 ... -0.03815857 -0.04307037
  -0.02108878]
 ...
 [-3.12326766  0.06489846  2.54482466 ... -0.02004622 -0.20608945
  -0.02152788

[[-4.17246771 -0.47828481 -0.04245704 ... -0.01393831 -0.02969191
  -0.02698778]
 [-0.25347611 -0.19785048  3.45937614 ...  0.80762337  0.16302155
   0.19445388]
 [ 8.74937151 -3.06166049  0.80849756 ...  0.59243746 -0.44951332
   0.11749992]
 ...
 [-2.76159197 -0.39213109 -0.36652287 ... -0.02873252 -0.18864149
  -0.03739102]
 [-4.01580414 -0.15040205 -0.13258164 ...  0.09723722 -0.12287922
   0.13207165]
 [-4.17246771 -0.47828481 -0.04245704 ... -0.01393831 -0.02969191
  -0.02698778]]
(1, 128, 32)
(30, 128, 32)
Index: 30
[[-3.97272028 -0.58312876 -0.34295805 ... -0.01378671 -0.02818714
   0.0113046 ]
 [-3.97272028 -0.58312876 -0.34295805 ... -0.01378671 -0.02818714
   0.0113046 ]
 [-3.97272028 -0.58312876 -0.34295805 ... -0.01378671 -0.02818714
   0.0113046 ]
 ...
 [-3.97071116 -0.58484623 -0.33985561 ... -0.01270734 -0.02883635
   0.00987571]
 [-3.97272028 -0.58312876 -0.34295805 ... -0.01378671 -0.02818714
   0.0113046 ]
 [-3.97272028 -0.58312876 -0.34295805 ... -0.01378671 -0.0281

[[-6.46534275  0.4858054   0.74713074 ...  0.00772769 -0.03757847
  -0.03053468]
 [-2.94338608 -0.5344341  -1.26625773 ...  0.02972423 -0.34058863
  -0.50244546]
 [ 5.39182831 -4.93826657 -0.34660954 ...  0.27580144  0.10376364
  -0.43610767]
 ...
 [-6.46534275  0.4858054   0.74713074 ...  0.00772769 -0.03757847
  -0.03053468]
 [-6.46534275  0.4858054   0.74713074 ...  0.00772769 -0.03757847
  -0.03053468]
 [-6.46534275  0.4858054   0.74713074 ...  0.00772769 -0.03757847
  -0.03053468]]
(1, 128, 32)
(45, 128, 32)
Index: 45
[[-4.21750872e+00 -7.86670801e-01  1.33657209e-01 ... -3.22493485e-02
  -6.98184346e-04  1.55447202e-02]
 [-4.21750872e+00 -7.86670801e-01  1.33657209e-01 ... -3.22493485e-02
  -6.98184346e-04  1.55447202e-02]
 [-4.21750872e+00 -7.86670801e-01  1.33657209e-01 ... -3.22493485e-02
  -6.98184346e-04  1.55447202e-02]
 ...
 [-3.17993790e+00  1.78010273e+00 -2.41680511e+00 ... -1.97963930e-01
   5.01582448e-01 -9.23736253e-02]
 [-4.08186441e+00 -6.80649076e-01 -3.50057244e

[[-5.1861383  -0.57966035  0.62431703 ... -0.01952026 -0.01322575
  -0.04354903]
 [-5.1861383  -0.57966035  0.62431703 ... -0.01952026 -0.01322575
  -0.04354903]
 [-5.1861383  -0.57966035  0.62431703 ... -0.01952026 -0.01322575
  -0.04354903]
 ...
 [-3.73658565  1.46595068  2.66883025 ...  0.07070958 -0.08145826
  -0.15310002]
 [-4.62578653  0.33484929  1.39633568 ... -0.06998446  0.14351323
  -0.15716855]
 [-5.1861383  -0.57966035  0.62431703 ... -0.01952026 -0.01322575
  -0.04354903]]
(1, 128, 32)
(59, 128, 32)
Index: 59
[[-6.11563249 -0.943693    0.35268236 ...  0.0954583  -0.01100915
   0.03797648]
 [-2.26564157 -2.4536316  -0.93660797 ... -0.04087789 -0.45728132
  -0.20815745]
 [ 6.71105477 -2.87111345  2.4988339  ...  0.0728164   0.04702653
   0.20458255]
 ...
 [-4.43449723  2.91923002  1.55144163 ... -0.01787098 -0.17970592
  -0.05583005]
 [-5.00799387  0.36193161  1.9182053  ...  0.00992168  0.05322693
   0.2387234 ]
 [-6.11563249 -0.943693    0.35268236 ...  0.0954583  -0.0110

...

[[-3.67122408 -0.25823121 -0.35006316 ... -0.01683285 -0.07690546
  -0.01897232]
 [-3.67122408 -0.25823121 -0.35006316 ... -0.01683285 -0.07690546
  -0.01897232]
 [-3.67122408 -0.25823121 -0.35006316 ... -0.01683285 -0.07690546
  -0.01897232]
 ...
 [-2.96391289 -0.65337525 -0.49046024 ...  0.14827697  0.75756187
   0.37072793]
 [-2.92038923 -0.65660506 -0.74836844 ... -0.14487804 -0.18787525
  -0.688285  ]
 [-3.44561835 -0.41161461 -0.52304606 ...  0.05879679 -0.19976965
   0.09428727]]
(1, 128, 32)
(256, 128, 32)
Index: 256
[[-3.82522133e+00 -4.90526194e-01  4.28081923e-01 ... -4.07426899e-02
  -5.63105902e-02  3.98042708e-03]
 [-8.92301919e-01  2.67098811e-01 -1.67322542e+00 ...  8.65762516e-01
  -4.33678930e-01  3.49627380e-02]
 [ 6.78514410e+00 -1.40195811e+00 -1.95952576e+00 ...  2.62502770e-02
  -9.52615833e-02  2.60749089e-01]
 ...
 [-3.09510435e+00  3.52564679e-01 -1.03013068e+00 ...  2.09466486e-01
  -6.70677641e-01 -3.52463282e-02]
 [-3.14799250e+00  3.52255323e-01 -9.9608992

[[-4.41520029e+00 -5.87586993e-01 -6.13554570e-01 ...  3.38721638e-02
   6.77335596e-03 -3.08555396e-03]
 [-5.36977137e-01 -7.83757132e-01 -2.87508772e+00 ...  2.43066830e-01
  -9.21058524e-02 -5.56685514e-01]
 [ 8.26348528e+00  2.73997684e+00 -1.97035859e+00 ... -8.62085895e-02
  -1.47546772e-01 -1.92388895e-01]
 ...
 [-4.05101447e+00 -6.61070423e-01 -8.40814253e-01 ...  4.49182681e-02
  -2.13347063e-02 -9.20286508e-03]
 [-4.41520029e+00 -5.87586993e-01 -6.13554570e-01 ...  3.38721638e-02
   6.77335596e-03 -3.08555396e-03]
 [-4.41520029e+00 -5.87586993e-01 -6.13554570e-01 ...  3.38721638e-02
   6.77335596e-03 -3.08555396e-03]]
(1, 128, 32)
(270, 128, 32)
Index: 270
[[-3.82693313 -0.37435624 -0.63946322 ... -0.03952569 -0.01865505
  -0.02387181]
 [-3.82693313 -0.37435624 -0.63946322 ... -0.03952569 -0.01865505
  -0.02387181]
 [-3.82693313 -0.37435624 -0.63946322 ... -0.03952569 -0.01865505
  -0.02387181]
 ...
 [-2.78346189  0.83113625  0.46888038 ...  0.04563959  0.40743063
   0.067539

[[-3.98230369 -0.85562231 -0.18729421 ... -0.03288282  0.00793995
   0.02124439]
 [-3.98230369 -0.85562231 -0.18729421 ... -0.03288282  0.00793995
   0.02124439]
 [-3.98230369 -0.85562231 -0.18729421 ... -0.03288282  0.00793995
   0.02124439]
 ...
 [-3.30511558 -0.3304381  -0.73748637 ...  0.05453526  0.05856552
  -0.36798351]
 [-3.91039567 -0.84256986 -0.22226247 ... -0.01844041 -0.01081096
   0.00762309]
 [-3.98230369 -0.85562231 -0.18729421 ... -0.03288282  0.00793995
   0.02124439]]
(1, 128, 32)
(284, 128, 32)
Index: 284
[[-4.54987492e+00 -1.91955451e-01 -7.79491563e-03 ...  2.78529703e-03
  -3.12330634e-02 -3.67679675e-02]
 [-6.82905631e-01 -7.25941273e-03 -3.12782561e+00 ...  8.79929588e-03
   1.76108439e-01 -1.31636444e-01]
 [ 8.25275363e+00 -3.20689885e+00 -6.55737378e-01 ... -2.62243922e-01
  -6.97531389e-02 -2.54490836e-01]
 ...
 [-4.00510286e+00 -3.21930870e-01 -3.24926797e-01 ... -5.90198832e-02
  -9.62445187e-02  1.31275113e-01]
 [-4.54987492e+00 -1.91955451e-01 -7.7949156

[[-4.41673195e+00  1.28929783e-01 -1.01931924e+00 ... -2.26442246e-02
  -2.46812945e-02 -5.00810751e-03]
 [-2.20568374e+00 -8.54643086e-01 -2.04504847e+00 ...  4.26693085e-01
   2.80072481e-01 -1.86006941e-02]
 [ 6.29649249e+00 -4.06639198e+00 -8.99201469e-01 ... -5.84549761e-01
  -1.35670772e-01  3.74762372e-01]
 ...
 [-4.41673195e+00  1.28929783e-01 -1.01931924e+00 ... -2.26442246e-02
  -2.46812945e-02 -5.00810751e-03]
 [-4.41673195e+00  1.28929783e-01 -1.01931924e+00 ... -2.26442246e-02
  -2.46812945e-02 -5.00810751e-03]
 [-4.41673195e+00  1.28929783e-01 -1.01931924e+00 ... -2.26442246e-02
  -2.46812945e-02 -5.00810751e-03]]
(1, 128, 32)
(298, 128, 32)
Index: 298
[[-4.68263744 -0.52154113  0.23188931 ... -0.01622342 -0.00787619
   0.03099247]
 [-4.68263744 -0.52154113  0.23188931 ... -0.01622342 -0.00787619
   0.03099247]
 [-4.68263744 -0.52154113  0.23188931 ... -0.01622342 -0.00787619
   0.03099247]
 ...
 [-4.68263744 -0.52154113  0.23188931 ... -0.01622342 -0.00787619
   0.030992

[[-4.43652592e+00 -3.41616627e-01 -2.71494008e-01 ...  7.28351048e-03
  -6.12175193e-02  1.00842604e-02]
 [-1.61057681e+00  8.29902817e-01 -1.80503122e+00 ... -6.87819914e-01
   3.53482856e-01 -1.26410983e-03]
 [ 5.98257527e+00  2.24371311e-01 -2.50720641e+00 ... -5.09097045e-01
   1.28942230e-01  4.59599832e-02]
 ...
 [-4.09392062e+00 -3.52270002e-01  1.01551947e-01 ...  2.83613494e-02
   8.39479389e-02 -2.38376582e-01]
 [-4.43652592e+00 -3.41616627e-01 -2.71494008e-01 ...  7.28351048e-03
  -6.12175193e-02  1.00842604e-02]
 [-4.43652592e+00 -3.41616627e-01 -2.71494008e-01 ...  7.28351048e-03
  -6.12175193e-02  1.00842604e-02]]
(1, 128, 32)
(312, 128, 32)
Index: 312
[[-4.25916172e+00 -4.93327980e-01 -5.06500456e-01 ...  3.36877407e-03
   9.41955646e-03 -2.01396506e-02]
 [-1.94135875e+00 -9.42040947e-01 -2.73645195e+00 ... -2.42524740e-01
  -1.57739686e-01 -1.88016657e-01]
 [ 6.94418942e+00 -2.48866347e+00 -1.35472812e+00 ... -2.45932413e-01
   3.34742585e-01  5.52583875e-01]
 ...
 [-3.

In [1]:
print(num_models)

NameError: name 'num_models' is not defined

In [54]:
from sklearn.cluster import KMeans

In [55]:
all_images[0].shape

(128, 32)

In [56]:
kmeans = KMeans(n_clusters=5,init='k-means++',random_state=0).fit(all_images[0])

In [57]:
kmeans.cluster_centers_

array([[ 2.90179656e-01, -1.39920149e+00,  1.48024268e+00,
        -2.53115236e-02,  4.72838166e-01,  2.65615188e-01,
        -2.61106102e-03, -2.05616027e-01,  9.37871109e-02,
         2.63078852e-01, -8.71713159e-02,  6.85010306e-02,
        -1.18728297e-01,  8.79544579e-02, -1.08004472e-02,
         1.91897760e-02, -7.02752307e-02, -1.11770959e-03,
         4.68062295e-02,  5.61552582e-02,  9.94672313e-03,
        -1.07330396e-02, -2.12430517e-03, -1.89294802e-02,
         1.01572089e-02, -5.84278725e-02,  2.32818704e-02,
        -6.03643154e-02, -1.47166816e-02, -5.79355862e-03,
         2.38277112e-02, -9.65641843e-04],
       [ 6.47851690e+00, -1.53766291e+00, -1.34871671e+00,
        -9.00208258e-01,  2.32364347e-01, -1.96108317e-01,
         1.32832045e-01,  7.51277199e-02,  7.47793653e-02,
         3.12937223e-02,  5.10705130e-02,  6.50006111e-02,
         4.67088545e-02,  1.77490712e-02, -4.71710433e-02,
         4.23352468e-02, -4.13908141e-02, -1.98310751e-02,
         3.04

In [58]:
kmeans.labels_

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 4, 4, 4, 1, 1, 1, 1,
       1, 4, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 4, 4, 4, 1, 1, 1, 1, 4,
       4, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 4, 4, 1, 1, 1, 1, 1, 4, 4,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

### Group by campose and then by KMeans

In [59]:
pca_dict = {}
del_keys = []
for key in images_dict:
        if not images_dict[key].any():
            print("No objects grouped under group",key)
            del_keys.append(key)

No objects grouped under group 1
No objects grouped under group 3
No objects grouped under group 5
No objects grouped under group 7


In [60]:
for k in del_keys:
    del images_dict[k]

In [26]:
images_dict[0].shape

(103, 3, 128, 128)

In [62]:
out_all_pca_grp_images = np.zeros((108,3,128,128))
for key in images_dict:
                images_val = images_dict[key].reshape(images_dict[key].shape[0],-1)
                
                pca = PCA(32)
                pca_image = pca.fit_transform(images_val)
                
                pca_image.reshape((-1,32,1,1))
                print(pca_image.shape)
                pca_dict[key] = pca_image
                #out_all_pca_grp_images[key]=np.concatenate((out_all_pca_grp_images[key],np.expand_dims(all_images[k],0)),axis=0)
                kmeans = KMeans(n_clusters=5,init='k-means++',random_state=0).fit(pca_image)
                print(kmeans.cluster_centers_)
                print(kmeans.labels_)

(100, 32)
[[-1.22645487e+01  3.37118093e-01  6.44534058e-01  1.96980377e-02
   4.20255536e-01  8.77756266e-01 -3.10395495e-01  2.65745086e-01
   2.28066320e-01 -1.33423715e-01  4.04761255e-02  2.08214424e-01
   7.17697973e-02 -2.56045212e-01  4.08066929e-02 -2.04939769e-01
  -8.88892287e-02 -1.46660991e-01 -5.64076900e-01  1.82388932e-01
   5.44236380e-01  1.87377759e-01 -4.12032457e-01 -3.48572344e-01
   2.90207199e-01  7.67353672e-02 -1.35348841e-01 -1.14540337e-01
  -1.74841375e-02 -3.04402666e-01 -6.91130523e-02 -2.48006902e-01]
 [ 1.69248698e+01 -1.48744197e+01  2.40591898e+00  4.15635489e+00
  -2.91373717e+00 -1.70162608e+00  7.64213668e-01 -1.60650728e+00
  -6.54314633e-01  1.30391540e-02  1.08323011e+00  6.60353560e-01
  -9.04673332e-02  4.72813782e-01 -9.28982771e-01  6.22171389e-01
  -1.63300439e-02  9.62737849e-01  6.06066685e-01  9.70918085e-02
  -6.32253826e-01 -9.49002576e-01  4.82681502e-02  8.02704905e-01
   8.16133147e-02  7.26853002e-01  9.30201746e-01  1.83564827e-01

(61, 32)
[[ 7.53170265e+00  1.17257829e+01 -3.91875447e+00  6.31059716e-01
   1.79216713e+00  1.87929524e+00  2.04712563e+00 -2.78301326e+00
   1.40785132e+00  5.44764543e-01 -2.54616995e+00 -7.69887056e-01
   1.78836067e+00 -1.49263237e-01 -4.92721280e-01 -1.21366101e+00
   7.11891902e-01 -5.18498589e-01 -1.02638549e+00  5.49982785e-01
  -7.49153919e-01  3.50555070e-01  9.97297890e-02 -1.26527484e+00
  -1.27283438e-01 -6.57588052e-01 -4.49734563e-02 -1.17421608e+00
  -8.77260896e-01  8.10334477e-01  4.40496418e-02  1.58288584e-02]
 [-6.85344133e+00 -4.50335947e+00  7.05300800e-01 -1.30902910e+00
   2.37841613e-01 -7.77169603e-01 -2.82043280e-01  1.97091482e-01
  -6.27075402e-01 -5.59095549e-01  4.28085204e-01  4.48984056e-01
  -6.37607898e-01  1.54734339e-01 -7.49305483e-02 -1.01047616e-01
  -2.47900059e-02  3.35341536e-01  2.76818273e-01 -4.13287860e-02
   1.92944168e-01  7.83095980e-02 -5.94031927e-02  3.47682868e-01
   7.35430684e-02  1.68111822e-01 -2.95645083e-02  2.66323165e-01
